In [17]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [18]:
batch_size = 64

train_root = r'C:\Users\Dhruv\Documents\Github\EE541\EE541_project\Preprocessing\symlink_Data\asl_alphabet_train'
train_set = torchvision.datasets.ImageFolder(train_root, transform = transforms.Compose([transforms.ToTensor()]))
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

In [19]:
print(train_set)

Dataset ImageFolder
    Number of datapoints: 78000
    Root location: C:\Users\Dhruv\Documents\Github\EE541\EE541_project\Preprocessing\symlink_Data\asl_alphabet_train
    StandardTransform
Transform: Compose(
               ToTensor()
           )


In [20]:
train_set[1][0].shape

torch.Size([3, 200, 200])

In [21]:
print("Class names = " + str(train_set.classes))

noc = 3
image_size_x = 200
image_size_y = 200

Class names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [22]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [23]:
input_units = noc*image_size_x*image_size_y
output_units = len(train_set.classes)

model1 = torch.nn.Sequential(nn.Flatten(),
                            nn.Linear(in_features=input_units,out_features=128),
                            nn.ReLU(),
                            nn.Linear(in_features=128,out_features=output_units))

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model1.parameters(),lr = 0.0001)


from torchsummary import summary
summary(model1,input_size=(1,3,200*200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1               [-1, 120000]               0
            Linear-2                  [-1, 128]      15,360,128
              ReLU-3                  [-1, 128]               0
            Linear-4                   [-1, 26]           3,354
Total params: 15,363,482
Trainable params: 15,363,482
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 0.92
Params size (MB): 58.61
Estimated Total Size (MB): 59.98
----------------------------------------------------------------


In [24]:
epochs = 40
size = len(train_loader.dataset)
train_loss = []
val_loss = []
train_acc = []
val_acc = []

for epoch in range(epochs):
    for batch, (x,y) in enumerate(train_loader):
        model1.train()
        
        x, y = x.to(device), y.to(device)
        y_pred = model1(x)
        loss = loss_func(y_pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 200 == 0:
        
            acc = (y_pred.argmax(1) == y).type(torch.float)
            acc = torch.mean(acc)
            
            train_loss.append(loss.item())
            train_acc.append(acc.item())

    loss = loss.item()
    print(f"Epoch = {epoch}, train loss: {loss:>7f}, train accuracy: {acc}")

Epoch = 0, train loss: 3.004816, train accuracy: 0.140625
Epoch = 1, train loss: 2.656423, train accuracy: 0.15625
Epoch = 2, train loss: 2.497186, train accuracy: 0.234375
Epoch = 3, train loss: 2.578449, train accuracy: 0.21875
Epoch = 4, train loss: 2.123648, train accuracy: 0.265625
Epoch = 5, train loss: 2.038876, train accuracy: 0.25
Epoch = 6, train loss: 2.142264, train accuracy: 0.203125
Epoch = 7, train loss: 1.997578, train accuracy: 0.203125
Epoch = 8, train loss: 2.171393, train accuracy: 0.328125
Epoch = 9, train loss: 2.045383, train accuracy: 0.375
Epoch = 10, train loss: 1.898989, train accuracy: 0.203125
Epoch = 11, train loss: 1.843387, train accuracy: 0.265625
Epoch = 12, train loss: 1.978200, train accuracy: 0.34375
Epoch = 13, train loss: 1.949866, train accuracy: 0.34375


KeyboardInterrupt: 